In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [5]:
sub = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [6]:
sub.head()

,row_id,num_sold
0,70128,100
1,70129,100
2,70130,100
3,70131,100
4,70132,100


In [7]:
test.head()

,row_id,date,country,store,product
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques


In [8]:
train.head()

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240


In [9]:
train['country'].unique()

array(['Belgium', 'France', 'Germany', 'Italy', 'Poland', 'Spain'],
      dtype=object)

In [10]:
train['store'].unique()

array(['KaggleMart', 'KaggleRama'], dtype=object)

In [11]:
train['product'].unique()

array(['Kaggle Advanced Techniques', 'Kaggle Getting Started',
       'Kaggle Recipe Book', 'Kaggle for Kids: One Smart Goose'],
      dtype=object)

In [12]:
train['date'].unique()

array(['2017-01-01', '2017-01-02', '2017-01-03', ..., '2020-12-29',
       '2020-12-30', '2020-12-31'], dtype=object)

In [13]:
train.head()

,row_id,date,country,store,product,num_sold
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240


In [41]:
train["country"] = train["country"].astype("category")
train["store"] = train["store"].astype("category")
train["product"] = train["product"].astype("category")

In [42]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()

In [43]:
y, X = train['num_sold'], train.drop('num_sold', axis=1)

In [32]:
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    break

TRAIN: [    0     1     2 ... 11685 11686 11687] TEST: [11688 11689 11690 ... 23373 23374 23375]


In [39]:
X_train

,row_id,date,country,store,product
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques
...,...,...,...,...,...
11683,11683,2017-09-01,Germany,KaggleMart,Kaggle for Kids: One Smart Goose
11684,11684,2017-09-01,Germany,KaggleRama,Kaggle Advanced Techniques
11685,11685,2017-09-01,Germany,KaggleRama,Kaggle Getting Started
11686,11686,2017-09-01,Germany,KaggleRama,Kaggle Recipe Book


In [49]:
X_train = pd.get_dummies(X_train, columns=['country', 'store', 'product'])
X_test = pd.get_dummies(X_test, columns=['country', 'store', 'product'])

In [56]:
# # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
# clf = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)
# # X is the dataframe we created in previous snippet
# clf.fit(X_train, y_train)
# # Must use JSON/UBJSON for serialization, otherwise the information is lost.
# clf.save_model("categorical-model.json")

In [57]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11688 entries, 0 to 11687
Data columns (total 14 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   row_id                                    11688 non-null  int64 
 1   date                                      11688 non-null  object
 2   country_Belgium                           11688 non-null  uint8 
 3   country_France                            11688 non-null  uint8 
 4   country_Germany                           11688 non-null  uint8 
 5   country_Italy                             11688 non-null  uint8 
 6   country_Poland                            11688 non-null  uint8 
 7   country_Spain                             11688 non-null  uint8 
 8   store_KaggleMart                          11688 non-null  uint8 
 9   store_KaggleRama                          11688 non-null  uint8 
 10  product_Kaggle Advanced Techniques        1168

In [58]:
X_train['date'] pd.to_datetime('date')

SyntaxError: invalid syntax (1552339661.py, line 1)

In [52]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)
param = {
    'max_depth': 2,
    'eta': 1,
    'nthread': 4,
    'eval_metric': 'auc'
}

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:date: object

In [ ]:
num_round = 10
model = xgb.train(param, dtrain, num_round)

In [ ]:
ypred = model.predict(dtest)

In [ ]:
xgb.plot_importance(model)

In [ ]:
dtrain.num_col()